In [12]:
import tensorflow as tf
import pickle
import numpy as np
import time
import load_cifar
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [13]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [26]:
num_classes = 10
num_features = 3072
num_output = 10
hidden_1 = 300
hidden_2 = 300
hidden_3 = 300
hidden_4 = 300
hidden_5 = 300
hidden_6 = 300
lr = 0.001
batch_size = 32
epochs = 50
dropout_prob = 0.5

<h1>Define Placeholder</h1>

In [15]:
X = tf.placeholder('float32',shape =(None,num_features),name="X")
Y = tf.placeholder('float32',shape = (None,num_classes),name="Y")

<h1>Define Neural Network Architecture</h1>

In [16]:
weights = {
    'W1': tf.Variable(tf.random_normal([num_features, hidden_1],stddev=0.1),name='W1'),
    'W2': tf.Variable(tf.random_normal([hidden_1, hidden_2],stddev=0.1),name='W2'),
    'W3': tf.Variable(tf.random_normal([hidden_2, hidden_3],stddev=0.1), name='W3'),
    'W4': tf.Variable(tf.random_normal([hidden_3, hidden_4],stddev=0.1), name='W4'),
    'W5': tf.Variable(tf.random_normal([hidden_4, hidden_5],stddev=0.1), name='W5'),
    'W6': tf.Variable(tf.random_normal([hidden_5, hidden_6],stddev=0.1), name='W6'),
    'Wout': tf.Variable(tf.random_normal([hidden_6, num_classes],stddev=0.1),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[hidden_2]),name='b2'),
    'b3': tf.Variable(tf.zeros(shape=[hidden_3]),name='b3'),
    'b4': tf.Variable(tf.zeros(shape=[hidden_4]),name='b4'),
    'b5': tf.Variable(tf.zeros(shape=[hidden_5]), name='b5'),
    'b6': tf.Variable(tf.zeros(shape=[hidden_6]), name='b6'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}

def neural_net(x):
    ly_1_out = tf.nn.relu(tf.add(tf.matmul(x,weights['W1']),biases['b1']))
    ly_2_out = tf.nn.relu(tf.add(tf.matmul(ly_1_out,weights['W2']),biases['b2']))
    ly_3_out = tf.nn.relu(tf.add(tf.matmul(ly_2_out, weights['W3']), biases['b3']))
    ly_4_out = tf.nn.relu(tf.add(tf.matmul(ly_3_out, weights['W4']), biases['b4']))
    ly_5_out = tf.nn.relu(tf.add(tf.matmul(ly_4_out, weights['W5']), biases['b5']))
    ly_6_out = tf.nn.relu(tf.add(tf.matmul(ly_5_out, weights['W6']), biases['b6']))
    out = tf.add(tf.matmul(ly_6_out,weights['Wout']),biases['bout'])
    return out


<h1>Define cost and optimization</h1>

In [17]:
logits = neural_net(X)
Y_hat = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

correct_pred = tf.equal(tf.argmax(Y_hat,1),tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [18]:
val_features, val_labels = pickle.load(open('vali_data.pickle','rb'))
training_features,training_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))
val_labels = np.squeeze(val_labels.todense())
training_labels = np.squeeze(training_labels.todense())
test_labels = np.squeeze(test_labels.todense())

In [19]:
def next_batch(batch_size, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    data_shuffle = np.asarray(data_shuffle)
    labels_shuffle = np.asarray(labels_shuffle)
    labels_shuffle = np.squeeze(labels_shuffle)
    return data_shuffle, labels_shuffle

In [27]:
init = tf.global_variables_initializer()

total_batch = int(40000 / batch_size)
num_steps = (epochs + 1) * total_batch

acc_val_list = []

with tf.Session() as sess:
    sess.run(init)
    time_start = time.time()
    for i in range(num_steps):
        # fetch batch
        batch_x, batch_y = next_batch(batch_size, training_features, training_labels)
        # run optimization
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if (i > 1) and (i % total_batch == 0):
            time_end = time.time()
            l, acc = sess.run([loss, accuracy], feed_dict={X: batch_x, Y: batch_y})
            acc_val = sess.run(accuracy, feed_dict={X: val_features, Y: val_labels})
            acc_val_list.append(acc_val)
            print("epoch " + str(int(i/total_batch)) + ", loss: {:.3f}".format(l) + \
                  ", acc: {:.3f}".format(acc) + ", acc_val:{:.3f}".format(acc_val) + \
                  ", time: {:.3f}\t".format(time_end-time_start))
            time_start = time.time()

    acc_test = sess.run(accuracy, feed_dict={X: test_features, Y: test_labels})
    print("acc_test: {:.3f}".format(acc_test))

epoch 1, loss: 1.625, acc: 0.438, acc_val:0.346, time: 5.572	
epoch 2, loss: 1.725, acc: 0.375, acc_val:0.372, time: 5.562	
epoch 3, loss: 1.253, acc: 0.562, acc_val:0.410, time: 5.704	
epoch 4, loss: 1.619, acc: 0.500, acc_val:0.415, time: 5.581	
epoch 5, loss: 1.387, acc: 0.406, acc_val:0.419, time: 5.679	
epoch 6, loss: 1.538, acc: 0.438, acc_val:0.445, time: 5.822	
epoch 7, loss: 1.370, acc: 0.594, acc_val:0.451, time: 5.666	
epoch 8, loss: 1.212, acc: 0.594, acc_val:0.448, time: 5.712	
epoch 9, loss: 1.393, acc: 0.594, acc_val:0.458, time: 5.770	
epoch 10, loss: 1.381, acc: 0.500, acc_val:0.470, time: 5.533	
epoch 11, loss: 1.231, acc: 0.500, acc_val:0.471, time: 5.755	
epoch 12, loss: 1.079, acc: 0.688, acc_val:0.474, time: 5.494	
epoch 13, loss: 1.470, acc: 0.531, acc_val:0.483, time: 5.433	
epoch 14, loss: 1.020, acc: 0.688, acc_val:0.495, time: 5.483	
epoch 15, loss: 1.446, acc: 0.500, acc_val:0.474, time: 5.401	
epoch 16, loss: 1.368, acc: 0.469, acc_val:0.497, time: 5.424	
e

Text(0, 0.5, 'Validation Accuracy (%)')

In [28]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(acc_val_list)), acc_val_list)
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")